# NERC with Fine-tuned Transformer Model using Large BERT 

We loaded the transformer model '[bert-large-NER](https://huggingface.co/dslim/bert-large-NER)' from the Hugging face repository, which is fine-tuned for Named Entity recognition.

The model is trained on the [CoNLL-2003](https://aclanthology.org/W03-0419/) database.

In [1]:
from simpletransformers.ner import NERModel
from sklearn.metrics import classification_report
import pandas as pd

/Users/sinemis/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Getting the test data into proper form for BERT

ner_test_data = pd.read_csv("NER-test.tsv", sep="\t")

test_sentences = []
current_sentence = []
for index, row in ner_test_data.iterrows():
    if row["token id"] == 0:
        if current_sentence:
            test_sentences.append(" ".join(current_sentence))
        current_sentence = []
    current_sentence.append(row["token"])

if current_sentence:
    test_sentences.append(" ".join(current_sentence))



In [3]:
# Getting the ground truth labels
ground_truth_labels = list(ner_test_data["BIO NER tag"])
print(ground_truth_labels)


['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-PERSON', 'O', 'O', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'I-WORK_OF_ART', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'I-WORK_OF_ART', 'I-WORK_OF_ART', 'I

In [4]:
# init NERModel
NAUR_model = NERModel(
    model_type="bert",
    model_name="dslim/bert-large-NER",
    use_cuda=False
)

Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# Predictions
predictions, _ = NAUR_model.predict(test_sentences)

print(predictions)

Running Prediction: 100%|██████████| 1/1 [00:02<00:00,  2.75s/it]

[[{'I': 'O'}, {'would': 'O'}, {"n't": 'O'}, {'be': 'O'}, {'caught': 'O'}, {'dead': 'O'}, {'watching': 'O'}, {'the': 'O'}, {'NFL': 'B-ORG'}, {'if': 'O'}, {'it': 'O'}, {'were': 'O'}, {"n't": 'O'}, {'for': 'O'}, {'Taylor': 'B-PER'}, {'Swift': 'I-PER'}, {'.': 'O'}], [{'Chris': 'B-PER'}, {"O'Donnell": 'I-PER'}, {'stated': 'O'}, {'that': 'O'}, {'while': 'O'}, {'filming': 'O'}, {'for': 'O'}, {'this': 'O'}, {'movie': 'O'}, {',': 'O'}, {'he': 'O'}, {'felt': 'O'}, {'like': 'O'}, {'he': 'O'}, {'was': 'O'}, {'in': 'O'}, {'a': 'O'}, {'Toys': 'B-MISC'}, {"''": 'I-ORG'}, {'R': 'I-MISC'}, {"''": 'I-ORG'}, {'Us': 'I-ORG'}, {'commercial': 'O'}, {'.': 'O'}], [{'The': 'O'}, {'whole': 'O'}, {'game': 'O'}, {'was': 'O'}, {'a': 'O'}, {'rollercoaster': 'O'}, {'ride': 'O'}, {',': 'O'}, {'but': 'O'}, {'Los': 'B-ORG'}, {'Angeles': 'I-ORG'}, {'Lakers': 'I-ORG'}, {'ultimately': 'O'}, {'persevered': 'O'}, {'and': 'O'}, {'won': 'O'}, {'!': 'O'}], [{'Zendaya': 'B-PER'}, {'slayed': 'O'}, {'in': 'O'}, {'Dune': 'B-MISC'}

In [9]:
# Need to get a list of only predicted labels for evaluation part
just_labels_list = []

for sentence_labels in predictions:
    for token_labels in sentence_labels:
        #print(token_labels)
        labella = next(iter(token_labels.values()))
        #print(only_value)
        just_labels_list.append(labella)


print(just_labels_list)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-ORG', 'I-MISC', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [10]:
# Evaluation
report = classification_report(ground_truth_labels, just_labels_list)
print(report)

               precision    recall  f1-score   support

       B-DATE       0.00      0.00      0.00         1
       B-MISC       0.00      0.00      0.00         0
        B-ORG       1.00      0.67      0.80         3
        B-PER       0.50      1.00      0.67         3
     B-PERSON       0.00      0.00      0.00         3
B-WORK_OF_ART       0.00      0.00      0.00         4
       I-DATE       0.00      0.00      0.00         1
       I-MISC       0.00      0.00      0.00         0
        I-ORG       1.00      0.83      0.91         6
        I-PER       0.33      1.00      0.50         1
     I-PERSON       0.00      0.00      0.00         2
I-WORK_OF_ART       0.00      0.00      0.00         9
            O       0.97      1.00      0.98       160

     accuracy                           0.89       193
    macro avg       0.29      0.35      0.30       193
 weighted avg       0.86      0.89      0.87       193



/Users/sinemis/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sinemis/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sinemis/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sinemis/anaconda3/lib/p